# 3.1.1.- Filtrado Colaborativo: *K-Nearest Neighbors*


* En esta notebook vamos a ver la ***técnica de los 'k-Vecinos' (KNN) aplicado a los sistemas de recomendación basados en filtrado colaborativo***.


<hr>


# K-Nearest Neighbors


* Los algoritmos de **filtrado colaborativo** basados en la técnica de los *k-Vecinos* (KNN) tratan de imitar el comportamiento de los seres humanos cuando buscan recibir una recomendación: cuando necesitamos conocer si nos va a interesar un item, preguntamos a personas que sabemos que conocen nuestros intereses si ellos consideran que el item nos va a gustar.


* Este método realizará esta misma operación empleando la matriz de votaciones. Este proceso seguirá el siguiente algoritmo:

    1. Determinar la ***similaridad entre los usuarios***.
    2. Encontrar el conjunto de ***k usuarios más similares (k-vecinos)***.
    3. ***Estimar las predicciones*** a los items no votados utilizando las votaciones realizadas por los *k* vecinos.
    4. (Opcional) ***Recomendar los 'N' items con la predicción más alta***.
    
    
* A modo esquemático tendríamos:


<img src="./../imgs/03_01_01_CF_Schema.png" style="width: 800px;"/>


* En los siguientes subapartados explicaremos en detalle cada uno de estos pasos:
<span></span><br>
    1. [Calculo de la similaridad](#M1)
<span></span><br>
    2. [Búsqueda de los *k-vecinos*](#M2)
<span></span><br>
    3. [Estimar las predicciones](#M3)
<span></span><br>
    4. [Cálculo de las recomendaciones](#M4)


* Y relizaremos un ejemplo en el implementaremos todo lo necesario para el cálculo de las predicciones y realización de las recomendaciones:
<span></span><br>
    5. [Ejemplo: Implementación de un Sistema de Recomendación con KNN](#M5)

<hr>

## <a name="M1">1.- Calculo de la similaridad</a>


* El primer paso del algoritmo de KNN consiste en ***determinar el parecido de cada pareja de usuarios basándonos en las votaciones*** previas de dichos usuarios. 


* Para calcular esta similaridad debemos tener presente que la ***matriz de votaciones es dispersa*** y, por lo tanto, ***parar comparar dos usuarios únicamente podremos emplear los votos sobre aquellos items que sean comunes a ambos usuarios***.


* Por ejemplo, si un usuario *u* ha realizado las siguientes votaciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|  1 	|  2 	|    	|  4 	|  2 	|    	|  3 	|    	|   	|   5  	|


* Y un usuario *v* ha realizado las siguientes votaciones:


|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| v 	|  3 	|   	|  2  	|   	|  5 	|  4  	|  3 	|  3  	|  1 	|    	|


* Únicamente podremos comparar los votos producidos en los items *i1*, *i5*, e *i7*.


* A continuación mostramos algunas métricas para calcular la similaridad entre usurios:

### - Métricas de Similaridad tradicionales: MSD, Coseno y Correlación de Pearson

* Existen infinidad de métricas de similaridad que permiten conocer el parecido de dos usuarios en función de sus votos comunes. Las más tradicionales se basan en medidas estadísticas clásicas, siendo las más conocidas y empleadas las siguientes:
<span></span><br><br>
    + ***MSD*** (*Mean Squared Difference| Diferencia Cuadrática Media*): Para que 1 sean la similaridad máxima y 0 la similaridad mínima el valor del MSD se invertirse ($1-MSD$) para lograr que cuanto más se parezcan dos usuarios, mayor sea su similaridad y viceversa.
<span></span><br><br>
$$sim(u,v)=1-MSD(u,v) = 1-\frac{1}{\#I_{u,v}}\sum_{i\in I_{u,v}}\left (  \frac{r_{u,i}-r_{v,i}}{max-min}\right )^2 \in \left [ 0,1 \right ]$$
<span></span><br>   
    + ***Coseno***:
$$sim(u,v) = \frac{\displaystyle \sum_{i\in I_{u,v}}r_{u,i} \cdot r_{v,i}}
{\sqrt{\displaystyle \sum_{i\in I_{u,v}}r_{u,i}^2} \cdot \sqrt{\displaystyle \sum_{i\in I_{u,v}}r_{v,i}^2}} \in \left [ 0,1 \right ]$$    
<span></span><br> 
    + ***Correlación de Pearson***:
<span></span><br><br>
$$sim(u,v) = \frac{\displaystyle \sum_{i\in I_{u,v}}(r_{u,i}-\bar{r_{u}})\cdot(r_{v,i}-\bar{r_{v}})}
{\sqrt{\displaystyle \sum_{i\in I_{u,v}}(r_{u,i}-\bar{r_{u})^2} \cdot \displaystyle \sum_{i\in I_{u,v}}(r_{v,i}-\bar{r_{v})^2}}} \in \left [ -1,1 \right ]$$


***NOTA***: *Al tener la Correlación valores de entre $[-1,1]$ deben de normalizarse estas similaridades por usuario para poder calcular posteriormente las predicciones.*


* En las formulas anteriormente propuestas tenemos:
    + $\#I_{u,v}$ representa el conjunto de los items que han votado tanto el usuario $u$ como el usuario $v$.
    + $max$ representa el valor del voto máximo.
    + $min$ representa el valor del voto mínimo.
    + $\bar{r}_u$ representa la votación media del usuario $u$.
    + $\bar{r}_v$ representa la votación media del usuario $v$.
    


### - Métricas de Similaridad Específicas: JMSD


* Aunque las métricas de similaridad basadas en medidas estadísticas clásicas ofrecen valores "aceptables" para el método de los *k-vecinos*, existen otras métricas de similaridad específicas para el dominio del filtrado colaborativo. 


* Una de las más populares es **JMSD** (Jaccard Mean Squared Difference) que permite calcular la similaridad atendiendo a dos factores: la información no numérica de los votos y la información numérica de los mismos. 


* *JMSD* se define como el producto del índice de Jaccard por la Diferencia Cuadrática Media:


$$JMSD(u,v) = Jaccard(u,v)  (1 - MSD(u, v))$$


* El ***índice de Jaccard es una medida que permite comparar la similitud de dos conjuntos***. En este caso se emplea para ***comprobar si los items votados por ambos usuarios coinciden***, independientemente de la votación:


$$Jaccard(u,v) =\frac {I_u \cap I_v} {I_u \cup I_v} = \frac {\# \{ i \in I | r_{u,i} \neq \bullet \wedge r_{v,i} \neq \bullet \}} {\# \{ i \in I | r_{u,i} \neq \bullet \vee r_{v,i} \neq \bullet \}}$$


* Esta medida busca un equilibrio entre el número de items que los dos usuarios han votado (Jaccard) y lo parecidas que son estas votaciones (MSD).


<hr>


## <a name="M2">2.- Búsqueda de los *k-vecinos*</a>


* En el método de KNN, las predicciones se realizan a partir del conjunto de k-vecinos de cada usuario, esto es, el conjunto de 'k' usuarios más similares a uno dado. 


* Este *k* será considerado como un parámetro del sistema y deberá "tunearse" para cada dataset o encontrar un *k* "óptimo". 


* ***La búsqueda de los k-vecinos consiste únicamente en ordenar los usuarios (vecinos) en base a su similaridad y elegir a los 'k' usuarios con la similaridad más alta***.



<hr>


## <a name="M3">3.- Estimación de las predicciones</a>


* La estimación de ***las predicciones se realiza agregando las votaciones que los k-vecinos del usuario activo que realizaron sobre el item que se quiere predecir***. 


* De nuevo, nos encontramos con el problema de la dispersión de la matriz de votaciones, ya que no todos los *k-vecinos* habrá votado los mismos items. Podría incluso darse la circunstancia de que un item no hubiera sido votado por ninguno de los *k-vecinos* y, por tanto, no podría estimarse una predicción. 


* Esta situación puede ayudarnos a "tunear" el parámetro *k* considerando los siguiente:
<span></span><br><br>
    + ***Si elegimos un k muy bajo*** conseguimos ***predicciones precisas*** ya que se realizar con usuarios muy similares al activo, pero ***existirán muchos items que no podremos predecir***.
<span></span><br><br>
    + ***Si elegimos un k muy elevado*** podremos ***predecir casi todos los items*** pero estas ***predicciones serán poco personalizadas***.
<span></span><br><br>
    + ***Si k es igual al número de usuario del sistema*** (menos uno), ***las predicciones*** serían equivalentes a la ***media de los votos***.
    
    
* Existen diversas formas de agregar las votaciones de los *k-vecinos* (*aggregation approach*), siendo las más utilizadas las siguientes 3:
    1. Media (***Average***) de los votos de los vecinos.
    2. Media ponderada (***Weighted Average***) de los votos de los vecinos (ponderada por su similaridad)
    3. Desviación respecto a la media (***Deviation From Mean***).
    
    
* Veamos a continuación la definición de estas formas de agregación:


### Average

* Esta es la aproximación más sencilla y simplemente cosiste en calcular la predicción como la ***media*** de los votos de los vecinos que han votado el item a predecir:

$$\hat{r}_{u,i} = \frac{1}{\#N_{u,i}} \sum_{n \in N_{u,i}} r_{n,i}$$


* Donde:
    + $N_{u,i}$ representa el conjunto de *k* vecinos del usuario $u$ que votaron el item $i$.


### Weighted Average


* Una evolución simple del cálculos de la media es la ***media ponderada***, en la cual el voto de cada uno de los vecinos se pondera en función de su similaridad con el usuario sobre el que se está calculando la predicción:


$$\hat{r}_{u,i} = \frac{\sum_{n \in N_{u,i}} sim(u,n) \cdot r_{n,i}}{\sum_{n \in N_{u,i}} sim(u,n)} $$


* Donde:
    + $N_{u,i}$ representa el conjunto de *k-vecinos* del usuario $u$ que votaron el item $i$.
    + $sim(u,n)$ simboliza la similaridad entre el usuario $u$ y el vecino $n$.


### Deviation From Mean


* Utilizar la media o la media ponderada como medida de agregación tiene un problema y es que presupone que todos los usuarios tienen la misma percepción de la escala de votaciones prefijada. Sin embargo, sabemos que esto no es cierto. 


* Existes determinados sesgos que hacen que los usuarios realicen votaciones haciendo una interpretación particular del sentido de su voto. Por ejemplo, existen usuarios más "generosos" con las votaciones que tienden a asignar siempre valoraciones altas y existen usuarios más "tacaños" con las votaciones que tienden a asignar siempre valoraciones más bajas. Que el primer usuario valore un item con 5 y el segundo usuario valore el mismo item con un 4 no quiere decir que al primero le haya gustado más el item. Cada usuario hace su propia interpretación de lo que significan los votos 4 y 5.


* Para incluir este fenómeno dentro de las medidas de agregación, es frecuente agregar las votaciones de los k vecinos mediante la ***desviación respecto a la media*** (*deviation from mean*):


$$\hat{r}_{u,i} = \bar{r}_{u} + \frac{ \sum_{n \in N_{u,i}} r_{n,i} - \bar{r}_n }{\#N_{u,i}}$$


* Donde:
    + $N_{u,i}$ representa el conjunto de *k* vecinos del usuario $u$ que votaron el item $i$
    + $\bar{r}_u$ representa la media de votos del usuario $u$
    + $\bar{r}_n$ representa la media de votos del usuario $n$.


* **CUIDADO**: *Cuando se calculan las predicciones con la desviación respecto a la media, puede darse el caso de que existan predicciones fuera del rango de notas; por tanto, deben de ajustarse las predicciones al valor de notas más cercano. Supongamos que la media de voto de un usuario es de 2,5 y que la desviación de voto de sus vecinos sobre un determinado item es de -2,2. En este caso la predicción de voto del item sería de 0.3. Al no poder votar un usuario un item con una nota inferior a 1 (y superior a 5), daremos como predicción a ese item la nota de 1*
    
<hr>


## <a name="M4">4.- Cálculo de las recomendaciones</a>


* El cálculo de las recomendaciones, por lo general, ***simplemente implica seleccionar los 'N' items con una predicción más alta***. Por ejemplo, si quisiéramos recomendar *N = 3* items a un usuario que tuviera las siguientes predicciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|   	|  2,9 	|    	|  4,7 	|  5,0 	|    	|  1,2 	|    	|   	|  3,1 	|


* Se le recomendarían a dicho usuario los items *i5*, *i4* e *i10*.


* En algunas ocasiones, es posible establecer filtros para acotar los items a recomendar. Por ejemplo: 

    + En un sistema de recomendación de restaurantes, es posible filtrar aquellos items que se encuentren a demasiada distancia del usuarios
    + En un sistema de recomendación de libros, el usuario puede filtrar el idioma o el género literario del libro
    + En una web de comercio electrónico es posible realizar recomendaciones sobre una categoría concreta...

<hr>


## <a name="M5">5.- Ejemplo: Implementación de un Sistema de Recomendación con KNN</a>


* A continuación vamos a realizar un ejemplo con fines didácticos en el que vamos a ir implementando paso por paso todo lo que hemos visto anteriormente que será todo lo necesario para construir un Sistema de Recomendación basado en Filtrado Colaborativo.


* Para ello veremos los siguientes puntos:
<span></span><br><br>
5.1. [Lectura del Dataset (Matriz de Votos)](#M51)
<span></span><br>
5.2. [Similaridades](#M52)
    <span></span><br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5.2.1 [Métricas de similaridad](#M521)
    <span></span><br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5.2.1.1 [MSD](#M5211)
    <span></span><br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5.2.1.2 [Coseno](#M5212)
    <span></span><br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5.2.1.3 [Correlación de Pearson](#M5213)
    <span></span><br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5.2.1.4 [JMSD](#M5214)
    <span></span><br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5.2.2 [Cálculo de similaridades](#M522)
<span></span><br>
5.3. [Búsqueda de los *k-vecinos*](#M53)
<span></span><br>
5.4. [Estimar las predicciones](#M54)
    <span></span><br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5.4.1 [Average](#M541)
    <span></span><br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5.4.2 [Weighted Average](#M542)
    <span></span><br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5.4.3 [Deviation From Mean](#M543)
<span></span><br>
5.5. [Cálculo de las recomendaciones](#M55)
<span></span><br>
5.6. [Evaluación del sistema de Recomendación](#M56)


### <a name="M51">5.1.- Lectura del Dataset (Matriz de Votos)</a>


* En primer lugar lo que necesitamos es un Dataset en el que se encuentren las votaciones que han realizado los usuarios sobre una serie de items.


* Para realiza este ejemplo con fines didácticos vamos a utilizar un Dataset con las siguientes características:
    + ***6 Usuarios***
    + ***12 Items***
    + ***Votos del 1 al 5***: Los usuarios pueden votar los items con las notas $\{ 1, 2, 3, 4, 5 \}$
    
    
* Con este Dataset vamos a ser capaces de construir una matriz de votos en el que las filas representarán a los usuarios y las columnas a los items.


* Vamos a implementar una función que dado un fichero con la estructura '*id_user::id_movie::rating*' nos devuelva la matriz de votos (una lista de listas), donde es las filas encontramos a los usuarios y en las columnas las películas. La ausencia de voto se representará con un None.

In [1]:
# Definimos dos constantes con el número de usuarios e items
NUM_USERS = 6
NUM_ITEMS = 12

def read_ratings_matrix(file):
    ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 
    
    with open(file, 'r') as reader:
        for line in reader:
            [u, i, rating] = line.split("::")
            ratings[int(u)][int(i)] = int(rating)
            
    return ratings

* Vamos a guardar la matriz de votos para utilizarla posteriormente y vamos a imprimirla por pantalla para ver la forma que tiene:

In [2]:
import numpy as np
import pandas as pd
import warnings

from tqdm.notebook import tqdm

pd.options.display.float_format = '{:,.2f}'.format
warnings.filterwarnings("ignore")

# Leemos el fichero y lo pasamos a una matriz (lista de listas)
RATINGS_FILE = './../data/db_prueba2.txt'
ratings_matrix = read_ratings_matrix(file=RATINGS_FILE)

# Mostramos la matriz de votos a modo informativo
pd.DataFrame(data=np.array([np.array(xi) for xi in ratings_matrix]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
U0,1,2,None,None,2,None,3,4,None,4,1,None
U1,None,None,1,5,None,5,3,1,None,5,2,1
U2,1,None,None,2,None,1,None,3,4,None,None,None
U3,None,1,4,4,None,None,3,None,5,4,None,1
U4,2,None,5,None,1,None,1,None,None,None,2,1
U5,None,None,5,2,1,None,None,4,None,1,None,2


### <a name="M52">5.2.- Similaridades</a>


* A continuación vamos a implementar las métricas de similaridad vistas anteriormente.


* Para hacer más amena la implementación vamos a suponer dos usuarios $u$ y $v$ con los siguientes votos que utilizaremos para calcular sus similaridades:


|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|  1 	|  2 	|    	|  4 	|  2 	|    	|  3 	|    	|   	|   5  	|
| v 	|  3 	|   	|  2  	|   	|  5 	|  4  	|  3 	|  3  	|  1 	|    	|


* Vemos que unicamente podremos comparar los votos producidos en los items *i1*, *i5*, e *i7* al ser los items que ambos usuarios han votado.

In [3]:
u = [1, 2, None, 4, 2, None, 3, None, None, 5]
v = [3, None, 2, None, 5, 4, 3, 3, 1, None]

* En primer lugar implementaremos una función auxiliar en el que dada las votaciones del usuario nos devolverá el valor medio de sus votos:

$$\bar{r}_u = \frac{1+2+4+2+3+5}{6} = \frac{17}{6} = 2,83$$

$$\bar{r}_v = \frac{3+2+5+4+3+3+1}{7} = \frac{21}{7} = 3,0$$

In [4]:
def rating_average(ratings):
    acc = 0
    count = 0
    for id_item in range(len(ratings)):
        if ratings[id_item] != None:
            acc += ratings[id_item]
            count += 1
    return acc / count

# Mostramos el valor medio de las votaciones de los usuarios u y v
print("AVG votos usuario u = {:0.2f}".format(rating_average(u)))
print("AVG votos usuario v = {:0.2f}".format(rating_average(v)))

AVG votos usuario u = 2.83
AVG votos usuario v = 3.00


#### <a name="M521">5.2.1- Métricas de similaridad</a>


* Tomando los votos de los usuarios $u$ y $v$ vamos a implementar y probar las métricas:
    + MSD
    + Coseno
    + Correlación de Pearson
    + JMSD


##### <a name="M5211">5.2.1.1- MSD</a>


* La similaridad usando el MSD viene dada por:


$$sim(u,v)=1-MSD(u,v) = 1-\frac{1}{\#I_{u,v}}\sum_{i\in I_{u}}\left (  \frac{r_{u,i}-r_{v,i}}{max-min}\right )^2 \in \left [ 0,1 \right ]$$


* Ejemplo cálculo similaridad Usuarios u y v:
<span></span><br><br>
    + $\#I_{u,v} = \{i1, i5, i7\} = 3$
<span></span><br><br>
    + $max = 5$
<span></span><br><br>
    + $min = 1$
    

$$sim(u,v) = 1 - \left (\frac{\left ( \frac{1-3}{5-1} \right )^2 + \left ( \frac{2-5}{5-1} \right )^2 + \left ( \frac{3-3}{5-1} \right )^2 }{3}  \right ) = 1- \left (\frac{0,25 + 0,56 + 0}{3} \right ) = 1 - 0,27 = 0,73$$


* Pasamos a implementar la métrica de similaridad:

In [5]:
import math

# Definimos dos constantes con los valores del voto máximo y mínimo
MAX_RATING = 5
MIN_RATING = 1

def msd_similarity(u, v):
    
    sum_r = 0 
    count = 0
    
    for i in range(len(u)):
        if u[i] != None and v[i] != None:
            count += 1
            sum_r += math.pow((u[i] - v[i])/(MAX_RATING - MIN_RATING), 2)
            
    if count > 0:
        sim = 1-(sum_r/float(count))
        return sim
    else:
        return None
            
            
# Mostramos el valor de la similaridad de los usuarios u y v
print("Similaridad MSD Usuarios u y v = {:0.2f}"
      .format(msd_similarity(u,v))) 

Similaridad MSD Usuarios u y v = 0.73


##### <a name="M5212">5.2.1.2- Coseno</a>


* El coseno viene dado por:

$$sim(u,v) = \frac{\displaystyle \sum_{i\in I_{u,v}}r_{u,i} \cdot r_{v,i}}
{\sqrt{\displaystyle \sum_{i\in I_{u,v}}r_{u,i}^2} \cdot \sqrt{\displaystyle \sum_{i\in I_{u,v}}r_{v,i}^2}} \in \left [ 0,1 \right ]$$


* Ejemplo cálculo similaridad Usuarios u y v:
<span></span><br><br>
    + $\#I_{u,v} = \{i1, i5, i7\} = 3$
    

$$sim(u,v) = \frac{(1\cdot3)+(2\cdot5)+(3\cdot3)}{\sqrt{1^2 + 2^2 + 3^2} \cdot \sqrt{3^2 + 5^2 + 3^2}} = \frac{3+10+9}{\sqrt{14} \cdot \sqrt{43}} = 0,90$$


* Pasamos a implementar la métrica de similaridad:

In [6]:
def cosine_similarity(u, v):
    
    numerador = 0
    denominador_u = 0
    denominador_v = 0
    
    count = 0
    
    for i in range(len(u)):
        if u[i] != None and v[i] != None:
            numerador += u[i] * v[i]
            denominador_u += math.pow(u[i], 2)
            denominador_v += math.pow(v[i], 2)
            count += 1
    if count > 0 and denominador_u != 0 and denominador_v != 0:
        cos = numerador / (math.sqrt(denominador_u) * math.sqrt(denominador_v))
        return cos
    else:
        return None
    
# Mostramos el valor de la similaridad de los usuarios u y v
print("Similaridad Coseno Usuarios u y v = {:0.2f}"
      .format(cosine_similarity(u,v))) 

Similaridad Coseno Usuarios u y v = 0.90


##### <a name="M5213">5.2.1.3- Correlación de Pearson</a>


* La correlación de Pearson viene dada por:


$$sim(u,v) = \frac{\displaystyle \sum_{i\in I_{u,v}}(r_{u,i}-\bar{r_{u}})\cdot(r_{v,i}-\bar{r_{v}})}
{\sqrt{\displaystyle \sum_{i\in I_{u,v}}(r_{u,i}-\bar{r_{u})^2} \cdot \displaystyle \sum_{i\in I_{u,v}}(r_{v,i}-\bar{r_{v})^2}}} \in \left [ -1,1 \right ]$$
<span></span><br><br>

* Ejemplo cálculo similaridad Usuarios u y v:
<span></span><br><br>
    + $\#I_{u,v} = \{i1, i5, i7\} = 3$
<span></span><br><br>
    + $\bar{r}_u = 2,83$
<span></span><br><br>
    + $\bar{r}_v = 3,00$
    
    
$$sim(u,v) = \frac{(1-2,83)\cdot(3-3) + (2-2,83)\cdot(5-3) + (3-2,83)\cdot(3-3)}{\sqrt{((1-2,83)^2+(2-2,83)^2+(3-2,83)^2) \cdot((3-3)^2+(5-3)^2+(3-3)^2)}} = \frac{-1,66}{\sqrt{4,07 \cdot 4,00}} = -0,41$$


* Pasamos a implementar la métrica de similaridad:

In [7]:
def correlation_similarity(u, v):
    
    numerador = 0
    denominador_u = 0
    denominador_v = 0
    
    count = 0
    avg_u = rating_average(u)
    avg_v = rating_average(v)
    
    for i in range(len(u)):
        if u[i] != None and v[i] != None:
            numerador += (u[i] - avg_u) * (v[i] - avg_v)
            denominador_u += math.pow((u[i] - avg_u), 2)
            denominador_v += math.pow((v[i] - avg_v), 2)
            count += 1
    if count > 0 and denominador_u != 0 and denominador_v != 0:
        corr = numerador / math.sqrt(denominador_u * denominador_v)
        return corr
    else:
        return None
    
# Mostramos el valor de la similaridad de los usuarios u y v
print("Similaridad Correlación de Pearson Usuarios u y v = {:0.2f}"
      .format(correlation_similarity(u,v))) 

Similaridad Correlación de Pearson Usuarios u y v = -0.41


##### <a name="M5214">5.2.1.4- JMSD</a>


* El JMSD viene dado por:


$$JMSD(u,v) = Jaccard(u,v) \cdot (1 - MSD(u, v)) = \frac {I_u \cap I_v} {I_u \cup I_v}  \cdot (1 - MSD(u, v))$$


* Ejemplo cálculo similaridad Usuarios u y v:
<span></span><br><br>
    + $I_u \cap I_v = \{i1, i5, i7\} = 3$
<span></span><br><br>
    + $I_u \cup I_v = \{i1, i2, i3, i4, i5, i6, i7, i8, i9, i10\} = 10$
<span></span><br><br>
    + $1-MSD(u,v) = 0,73$
    
$$JMSD(u,v) = \frac{3}{10} \cdot 0,73 = 0,22$$


* Pasamos a implementar la métrica de similaridad:

In [8]:
def jmsd_similarity(u, v):
    
    union = 0
    intersection = 0
    
    for i in range(len(u)):
        if u[i] != None and v[i] != None:
            intersection += 1
            union += 1
        elif u[i] != None or v[i] != None:  
            union += 1
            
    if intersection > 0:
        jaccard = intersection / union
        return jaccard * msd_similarity(u,v)
    else:
        return None

# Mostramos el valor de la similaridad de los usuarios u y v
print("Similaridad JMSD Usuarios u y v = {:0.2f}"
      .format(jmsd_similarity(u,v))) 

Similaridad JMSD Usuarios u y v = 0.22


#### <a name="M522">5.2.2- Cálculo de similaridades</a>


* Una vez que ya tenemos definidas las métricas de similaridad, procedemos a ***calcular las similaridades de todos los usuarios con todos***.


* El resultado de esta tarea será la de una ***matriz de dimensión "NUM USERS x NUM USERS" donde estarán calculadas las similaridades entre cada par de usuarios***.


* Los valores de la diagonal no nos interesan para posteriores cálculos ya que en ella se encontrará la similaridad que hay entre un mismo usuario y esta será máxima.


* A continuación creamos la función "calculate_similarities()" a la que pasaremos como parámetros la matriz de votos y una métrica de similaridad implementada anteriormente:

##### NOTA: Por comodidad para futuros cálculos, asignamos con valor menos infinito la similaridad sobre el mismo usuario.

In [9]:
def calculate_similarities(ratings_matrix, similarity_metric):
    
    # Creamos una matriz con valores de similaridad a -infinito
    similarities = [[float('-inf') for _ in range(NUM_USERS)] for _ in range(NUM_USERS)]
    
    # Recorremos la matriz por usuario
    for i, u in enumerate(tqdm(ratings_matrix, leave=False)):
        for j, v in enumerate(ratings_matrix):
            if j != i: # No calculamos la similaridad para un mismo usuario
                similarities[i][j] = similarity_metric(u,v)
        
    return similarities
    

# Vamos a realizar los cálculos de la similaridad usando el Coseno
similarities = calculate_similarities(ratings_matrix=ratings_matrix, similarity_metric=cosine_similarity) 

* Mostramos a continuación como quedaría la matriz de similaridades, en la que podemos ver como la matriz triangular superior e inferiore son simétricas:

In [10]:
pd.DataFrame(data=np.array([np.array(xi) for xi in similarities]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["U{}".format(str(i)) for i in range(NUM_USERS)])

,U0,U1,U2,U3,U4,U5
U0,-inf,0.86,1.00,0.98,0.73,0.86
U1,0.86,-inf,0.67,0.91,0.60,0.51
U2,1.00,0.67,-inf,0.98,1.00,0.99
U3,0.98,0.91,0.98,-inf,0.91,0.83
U4,0.73,0.60,1.00,0.91,-inf,0.98
U5,0.86,0.51,0.99,0.83,0.98,-inf


### <a name="M53">5.3.- Búsqueda de los *k-vecinos*</a>


* Conceptualmente la búsqueda de los *k-vecinos* es sencillo ya que consiste en ordenar por similaridad (de mayor a menor similaridad) a los vecinos del usuario activo y quedarnos con lo 'K' más similares.


* Computacionalmente hay que recordar que las tareas de ordenación son especialmente costosas, por lo que hay que tener cuidado como realizar estas ordenaciones.


* Por último tenemos que definir previamente un valor de 'K' para seleccionar el número de vecinos.


* Pasamos a implementar la funcion "calculate_neighbors()" al que le pasaremos la matriz de similaridades y el número de vecinos a calcular y nos devolverá una matriz de "*NUM_USERS x K-NEIGHBORS*", indicando en cada fila el ídentificador de los vecinos del usuario:


In [11]:
# Definimos como constante el número de 'K' vecinos
K = 2

def calculate_neighbors(similarities_matrix, k_neighbors):
    
    neighbors = [None for _ in range(NUM_USERS)]
    
    for index, similarities in enumerate(tqdm(similarities_matrix, leave=False)):
        i_neighbors = [i[0] for i in sorted(enumerate(similarities), 
                                            key=lambda x:float('-inf') if x[1] is None else x[1], 
                                            reverse=True)]
        neighbors[index] = i_neighbors[0:k_neighbors]

    return neighbors

neighbors = calculate_neighbors(similarities_matrix=similarities, k_neighbors=K)

* Veamos a continuación como quedan los K-Vecinos de los usuarios:

In [12]:
pd.DataFrame(data=np.array([np.array(xi) for xi in neighbors]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["K{}".format(str(i)) for i in range(K)])

,K0,K1
U0,2,3
U1,3,0
U2,4,0
U3,0,2
U4,2,5
U5,2,4


* Veamos que efectivamente las similaridades de los vecinos de decrecientes:

In [13]:
aux_sim = [[similarities[index_u][neighbord] for index_j, neighbord in enumerate(user)] 
           for index_u, user in enumerate(neighbors)]
pd.DataFrame(data=np.array([np.array(xi) for xi in aux_sim]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["K{}".format(str(i)) for i in range(K)])

,K0,K1
U0,1.00,0.98
U1,0.91,0.86
U2,1.00,1.00
U3,0.98,0.98
U4,1.00,0.98
U5,0.99,0.98


### <a name="M54">5.4.- Estimar las predicciones</a>


* A continuación vamos a implemantar los métodos de agregación de votaciones vistos anteriormente:

#### <a name="M541">5.4.1.- Average</a>


* Consiste en dar como predicción del voto, el valor medio del voto de los vecinos que hayan votado el item a recomendar:

$$\hat{r}_{u,i} = \frac{1}{\#N_{u,i}} \sum_{n \in N_{u,i}} r_{n,i}$$


* Pasamos a implementar la funcion "calculate_average_prediction()" al que le pasaremos la matriz de votos y los *K-vecinos* y nos devolverá una matriz de "*NUM_USERS x NUM_ITEMS*", con las predicciones:


In [14]:
def calculate_average_prediction(ratings_matrix, neighbors):
    
    # Creamos una matriz para el cálculo de predicciones
    predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]
    
    # Recorremos la matriz de votos
    for i, u in enumerate(tqdm(ratings_matrix, leave=False)):
        for j, v in enumerate(ratings_matrix[0]):

            # Obtenemos las votaciones de cada vecino y calculamos el voto medio
            sum_r = 0 
            count = 0
            for neighbor in neighbors[i]:
                if ratings_matrix[neighbor][j] != None:
                    count += 1
                    sum_r += ratings_matrix[neighbor][j]
                    
            predictions[i][j] = None if count == 0 else sum_r/count
            
    return predictions
            
predictions_avg = calculate_average_prediction(ratings_matrix=ratings_matrix, neighbors=neighbors)

* Mostramos a continuación como quedan las predicciones de los items, usando como método de agregación la media.

In [15]:
pd.DataFrame(data=np.array([np.array(xi) for xi in predictions_avg]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
U0,1.00,1.00,4.00,3.00,None,1.00,3.00,3.00,4.50,4.00,None,1.00
U1,1.00,1.50,4.00,4.00,2.00,None,3.00,4.00,5.00,4.00,1.00,1.00
U2,1.50,2.00,5.00,None,1.50,None,2.00,4.00,None,4.00,1.50,1.00
U3,1.00,2.00,None,2.00,2.00,1.00,3.00,3.50,4.00,4.00,1.00,None
U4,1.00,None,5.00,2.00,1.00,1.00,None,3.50,4.00,1.00,None,2.00
U5,1.50,None,5.00,2.00,1.00,1.00,1.00,3.00,4.00,None,2.00,1.00


#### <a name="M542">5.4.2.- Weighted Average</a>


* Consiste en dar como predicción del voto el voto medio de los vecinos que hayan votado el item a recomendar, ponderado su similaridad:


$$\hat{r}_{u,i} = \frac{\sum_{n \in N_{u,i}} sim(u,n) \cdot r_{n,i}}{\sum_{n \in N_{u,i}} sim(u,n)} $$


*  * Pasamos a implementar la funcion "calculate_average_prediction()" al que le pasaremos la matriz de votos y los *K-vecinos* y nos devolverá una matriz de "*NUM_USERS x NUM_ITEMS*", con las predicciones:

In [16]:
def calculate_weighted_average_prediction(ratings_matrix, similarities_matrix, neighbors):
    
    # Creamos una matriz para el cálculo de predicciones
    predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]
    
    # Recorremos la matriz de votos
    for i, u in enumerate(tqdm(ratings_matrix, leave=False)):
        for j, v in enumerate(ratings_matrix[0]):
            # Obtenemos las similaridades con cada vecino y si voto
            numerador = 0 
            denominador = 0
            for neighbor in neighbors[i]:
                if ratings_matrix[neighbor][j] != None:
                    numerador += similarities_matrix[i][neighbor] * ratings_matrix[neighbor][j]
                    denominador += similarities_matrix[i][neighbor]
                    
            predictions[i][j] = None if denominador == 0 else numerador/denominador
            
    return predictions
            
predictions_weighted_avg = calculate_weighted_average_prediction(ratings_matrix=ratings_matrix, 
                                                                 similarities_matrix=similarities, 
                                                                 neighbors=neighbors)

* Mostramos a continuación como quedan las predicciones de los items, usando como método de agregación la media poderada.

In [17]:
pd.DataFrame(data=np.array([np.array(xi) for xi in predictions_weighted_avg]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
U0,1.00,1.00,4.00,2.99,None,1.00,3.00,3.00,4.50,4.00,None,1.00
U1,1.00,1.49,4.00,4.00,2.00,None,3.00,4.00,5.00,4.00,1.00,1.00
U2,1.50,2.00,5.00,None,1.50,None,2.00,4.00,None,4.00,1.50,1.00
U3,1.00,2.00,None,2.00,2.00,1.00,3.00,3.50,4.00,4.00,1.00,None
U4,1.00,None,5.00,2.00,1.00,1.00,None,3.50,4.00,1.00,None,2.00
U5,1.50,None,5.00,2.00,1.00,1.00,1.00,3.00,4.00,None,2.00,1.00


#### <a name="M543">5.4.3.- Deviation From Mean</a>

* Consiste en dar como predicción el voto médio del usario, añadiendo un factor corrector que es como los vecinos del usuario votan el item a recomendar respecto a su votación media:


$$\hat{r}_{u,i} = \bar{r}_{u} + \frac{ \sum_{n \in N_{u,i}} r_{n,i} - \bar{r}_n }{\#N_{u,i}}$$

In [18]:
def calculate_deviation_from_mean_prediction(ratings_matrix, neighbors):
    
    # Creamos una matriz para el cálculo de predicciones
    predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]
    
    # Calculamos la media de los votos de los usuarios
    avg_user_ratings = [rating_average(user) for user in ratings_matrix]
    
    # Recorremos la matriz de votos
    for i, u in enumerate(tqdm(ratings_matrix, leave=False)):
        for j, v in enumerate(ratings_matrix[0]):
            numerador = 0 
            denominador = 0
            for neighbor in neighbors[i]:
                if ratings_matrix[neighbor][j] != None:
                    numerador += ratings_matrix[neighbor][j] - avg_user_ratings[neighbor]
                    denominador += 1
                    
            predictions[i][j] = (avg_user_ratings[i] if denominador == 0 
                                 else avg_user_ratings[i] + (numerador/denominador))
            
            # Ajustamos los votos inferiores a 1 y superiores a 5
            if predictions[i][j] < 1:
                predictions[i][j] = 1
            if predictions[i][j] > 5:
                predictions[i][j] = 5
            
    return predictions
    

predictions_deviation_from_mean = calculate_deviation_from_mean_prediction(ratings_matrix=ratings_matrix,
                                                                           neighbors=neighbors)

* Mostramos a continuación como quedan las predicciones de los items, usando como método de agregación la *desviación respecto a la media*:

In [19]:
pd.DataFrame(data=np.array([np.array(xi) for xi in predictions_deviation_from_mean]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
U0,1.23,1.00,3.29,2.76,2.43,1.23,2.29,3.23,4.26,3.29,2.43,1.00
U1,1.45,1.59,3.73,3.73,2.45,2.88,3.09,4.45,4.73,4.09,1.45,1.00
U2,1.49,1.77,5.00,2.20,1.49,2.20,1.99,3.77,2.20,3.77,1.49,1.20
U3,1.83,2.71,3.14,2.94,2.71,1.94,3.71,4.33,4.94,4.71,1.71,3.14
U4,1.00,2.00,4.50,1.65,1.00,1.00,2.00,3.15,3.80,1.00,2.00,1.50
U5,1.90,2.50,5.00,2.30,1.50,1.30,1.50,3.30,4.30,2.50,2.50,1.50


### <a name="M55">5.5.- Cálculo de las recomendaciones</a>


* Una vez que tenemos calculadas las predicciones tenemos que recomendar a cada usuario 'N' items que no ha valorado, ordenados por la nota de la predicción.


* Veamos a continuación como realizar la recomendación, implementando la función "make_recommendations()" que recibe como parámetros el número de recomendaciones a realizar, la matrix de votos y la matriz de predicciones y devuelve una matriz con los items a recomendar. Esta función recorrerá la matriz de votos, vera los items no votados y recomendaremos esos items a los usuarios ordenandolos por la predicción del voto, truncando el número de recomendaciones a las indicadas por la función:


In [20]:
N = 12

def make_recommendations(num_recomendations, ratings_matrix, predictions_matrix):
    
    # Creamos una matriz para las recomendaciones
    recommendations = [[(None, None) for _ in range(num_recomendations)] for _ in range(NUM_USERS)]
    
    # Recorremos la matriz de votos
    for i, u in enumerate(tqdm(ratings_matrix, leave=False)):
        for j, v in enumerate(ratings_matrix[0]):
            if ratings_matrix[i][j] == None:
                recommendations[i].append((j, predictions_matrix[i][j]))
        
        # Ordenamos los items a recomendar al usuario
        recommendations[i] = sorted(recommendations[i], 
                                    key=lambda x:float('-inf') if x[1] is None else x[1], 
                                    reverse=True)[0:num_recomendations]

    return [[x[0] for x in reco_user] for reco_user in recommendations]

                
recomendations = make_recommendations(num_recomendations=N,
                                      ratings_matrix=ratings_matrix,
                                      predictions_matrix=predictions_avg)

* Mostramos a continuación como quedan las recomendaciones a realizar a los usuarios (3 recomendaciones en caso de que se puedan realizar):

In [21]:
pd.DataFrame(data=np.array([np.array(xi) for xi in recomendations]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["Reco{}".format(str(i+1)) for i in range(N)])

,Reco1,Reco2,Reco3,Reco4,Reco5,Reco6,Reco7,Reco8,Reco9,Reco10,Reco11,Reco12
U0,8,2,3,5,11,None,None,None,None,None,None,None
U1,8,4,1,0,None,None,None,None,None,None,None,None
U2,2,9,1,6,4,10,11,None,None,None,None,None
U3,7,4,0,5,10,None,None,None,None,None,None,None
U4,8,7,3,5,9,None,None,None,None,None,None,None
U5,8,10,0,5,6,None,None,None,None,None,None,None


### <a name="M56">5.6.- Evaluación del sistema de Recomendación</a>


* Por último vamos a calcular el error medio (MAE) cometido en las predicciones realizadas.


* Al no dividir en este ejemplo dos datos en conjunto de entrenamiento y test, vamos a evaluar las predicciones sobre el dataset del ejemplo.


* Para ello vamos a implementar una función llamada "get_mae()" que recibirá como parámetros la matriz de votos y la matriz de predicciones y devolverá el MAE del Sistema de Recomendación.


* Recordar que definimos el MAE de un usuario como:
<span></span><br><br>
$$MAE_u = \frac{ \sum_{i \in I^T_u} \mid r_{u,i} - \hat{r}_{u,i} \mid  }{\#I^T_u} $$
<span></span><br><br>
    donde $I^T_u$ representa el conjunto de items de test votados por el usuario $u$.
<span></span><br><br>
* Definimos el *MAE* del sistema como el promedio del *MAE* de cada usuario:
<span></span><br><br>
$$MAE = \frac{ \sum_{u \in U^T} MAE_u }{ \#U^T } $$

In [22]:
def get_mae(ratings_matrix, predictions_matrix):
    
    mae_users = [None for _ in ratings_matrix]
    
    # Recorremos la matriz de votos
    for i, u in enumerate(tqdm(ratings_matrix, leave=False)):
        # Calculamos el MAE de cada usuario
        sum_user = 0
        count = 0
        for j, v in enumerate(ratings_matrix[0]):
            if ratings_matrix[i][j] != None and predictions_matrix[i][j] != None:
                sum_user += abs(ratings_matrix[i][j] - predictions_matrix[i][j])
                count += 1
        mae_users[i] = sum_user/count if count > 0 else None

    return np.nanmean(np.array(mae_users, dtype=np.float), axis=0)


mae_avg_predictions = get_mae(ratings_matrix=ratings_matrix, 
                              predictions_matrix=predictions_avg)

mae_weighted_avg = get_mae(ratings_matrix=ratings_matrix, 
                           predictions_matrix=predictions_weighted_avg)

mae_predictions_deviation_from_mean = get_mae(ratings_matrix=ratings_matrix, 
                                              predictions_matrix=predictions_deviation_from_mean)

print('MAE con avg_predictions: {:0.4f}'.format(mae_avg_predictions))
print('MAE con weighted_avg: {:0.4f}'.format(mae_weighted_avg))
print('MAE con mae_predictions_deviation_from_mean: {:0.4f}'.format(mae_predictions_deviation_from_mean))

MAE con avg_predictions: 0.6893
MAE con weighted_avg: 0.6893
MAE con mae_predictions_deviation_from_mean: 0.8595


<hr>


## Referencias

Bobadilla, J., Serradilla, F., & Bernal, J. (2010). **A new collaborative filtering metric that improves the behavior of recommender systems**. Knowledge-Based Systems, 23(6), 520-528.

<hr>


*Este documento ha sido desarrollado por **Ricardo Moya**, basandose en el material creado por **Fernando Ortega**. Dpto. Sistemas Informáticos, ETSI de Sistemas Informáticos, Universidad Politécnica de Madrid.* respetando la licencia: "Atribución-NoComercial-CompartirIgual" definida por **Creative Commons Corporation**.


<img src="../imgs/CC_BY-NC-SA.png" alt="CC BY-NC">

<p style="text-align:center"><b>Atribución-NoComercial-CompartirIgual</b></p>
